# Additional Cross-Validation Results for Prophet

We use monthly data, instad of annual data
We use US overall, instead of per country data





In [1]:
# LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

Importing plotly failed. Interactive plots will not work.


In [2]:
# DATA

df = pd.read_csv(r'tidy_dataset1_v1.csv')

In [3]:
df.tail()

,Date,Motor_vehicles_and_parts,Furnishings_and_durable_household_equipment,Recreational_goods_and_vehicles,Other_durable_goods,Food_and_beverages_purchased_for_off-premises_consumption,Clothing_and_footwear,Gasoline_and_other_energy_goods,Other_nondurable_goods,Housing_and_utilities,Health_care,Transportation_services,Recreation_services,Food_services_and_accommodations,Financial_services_and_insurance,Other_services,GDP,Disposable_personal_income,Population,CO2
295,2021-08-01,218756.67,164968.67,201490.33,93114.67,403910.67,157055.00,129187.33,470564.33,930793.67,871689.00,161493.33,183949.67,375100.33,429012.33,437171.67,7850140.00,18395900.0,332295000.0,414.47
296,2021-09-01,218756.67,164968.67,201490.33,93114.67,403910.67,157055.00,129187.33,470564.33,930793.67,871689.00,161493.33,183949.67,375100.33,429012.33,437171.67,7850140.00,18193900.0,332403000.0,413.30
297,2021-10-01,231653.67,166509.67,207863.67,94500.33,413007.33,157611.67,143548.00,476539.67,943142.67,883622.67,165667.33,193194.67,386395.33,437368.00,442490.33,8116373.67,18304200.0,332513000.0,413.93
298,2021-11-01,231653.67,166509.67,207863.67,94500.33,413007.33,157611.67,143548.00,476539.67,943142.67,883622.67,165667.33,193194.67,386395.33,437368.00,442490.33,8116373.67,18367800.0,332598000.0,415.01
299,2021-12-01,231653.67,166509.67,207863.67,94500.33,413007.33,157611.67,143548.00,476539.67,943142.67,883622.67,165667.33,193194.67,386395.33,437368.00,442490.33,8116373.67,18396200.0,332640000.0,416.71


In [4]:
# DATA 2

df['consumer_goods'] = df['Motor_vehicles_and_parts'] + df['Furnishings_and_durable_household_equipment'] 
+ df['Other_durable_goods'] + df['Clothing_and_footwear'] + df['Other_nondurable_goods']

df['leisure'] = df['Recreational_goods_and_vehicles'] + df['Recreation_services'] 
+ df['Food_services_and_accommodations']

df['nutrition'] = df['Food_and_beverages_purchased_for_off-premises_consumption']

df['mobility'] = df['Gasoline_and_other_energy_goods'] + df['Transportation_services']

df['housing'] = df['Housing_and_utilities']

df['services'] = df['Health_care'] + df['Financial_services_and_insurance'] + df['Other_services']

# drop original variables

df = df[['Date', 'CO2', 'consumer_goods', 'leisure', 'nutrition', 'mobility', 'housing', 'services']]

In [5]:
df.rename(columns = {'Date':'ds', 'CO2':'y'}, inplace = True)

In [6]:
df_split = df.copy()
df_fit = df_split[:-12]
df_fit.tail()

,ds,y,consumer_goods,leisure,nutrition,mobility,housing,services
283,2020-08-01,412.78,344247.67,322839.00,377545.00,198265.67,893986.00,1568723.0
284,2020-09-01,411.52,344247.67,322839.00,377545.00,198265.67,893986.00,1568723.0
285,2020-10-01,411.51,347336.00,325442.33,375898.67,202180.66,899999.33,1619435.0
286,2020-11-01,413.12,347336.00,325442.33,375898.67,202180.66,899999.33,1619435.0
287,2020-12-01,414.26,347336.00,325442.33,375898.67,202180.66,899999.33,1619435.0


In [7]:
df_predict = df_split[-12:]
df_predict.tail()

,ds,y,consumer_goods,leisure,nutrition,mobility,housing,services
295,2021-08-01,414.47,383725.34,385440.00,403910.67,290680.66,930793.67,1737873.0
296,2021-09-01,413.30,383725.34,385440.00,403910.67,290680.66,930793.67,1737873.0
297,2021-10-01,413.93,398163.34,401058.34,413007.33,309215.33,943142.67,1763481.0
298,2021-11-01,415.01,398163.34,401058.34,413007.33,309215.33,943142.67,1763481.0
299,2021-12-01,416.71,398163.34,401058.34,413007.33,309215.33,943142.67,1763481.0


In [8]:
model = Prophet(interval_width=0.95)

model.add_regressor('consumer_goods', standardize=True)
model.add_regressor('leisure', standardize=True)
model.add_regressor('nutrition', standardize=True)
model.add_regressor('mobility', standardize=True)
model.add_regressor('housing', standardize=True)
model.add_regressor('services', standardize=True)
    
model.fit(df_fit)

forecast = model.predict(df_predict)
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]

22:03:11 - cmdstanpy - INFO - Chain [1] start processing
22:03:11 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper
0,2021-01-01,416.104867,415.547554,416.633377
1,2021-02-01,416.954580,416.437239,417.518924
2,2021-03-01,417.831024,417.232201,418.378307
3,2021-04-01,419.505266,418.951026,420.055523
4,2021-05-01,420.112219,419.521562,420.732508
5,2021-06-01,419.437903,418.738205,420.103912
6,2021-07-01,417.946950,417.253868,418.665705
7,2021-08-01,415.986665,415.139461,416.829063
8,2021-09-01,414.559424,413.568829,415.447979
9,2021-10-01,415.121538,414.003854,416.234499


In [9]:
cv_results = cross_validation(
        model = model,
        initial = '732 days',
        period = '365 days',
        horizon = '366 days')
   
df_p = performance_metrics(cv_results)

  0%|          | 0/21 [00:00<?, ?it/s]

22:03:11 - cmdstanpy - INFO - Chain [1] start processing
22:03:32 - cmdstanpy - INFO - Chain [1] done processing
22:03:32 - cmdstanpy - INFO - Chain [1] start processing
22:03:43 - cmdstanpy - INFO - Chain [1] done processing
22:03:43 - cmdstanpy - INFO - Chain [1] start processing
22:03:43 - cmdstanpy - INFO - Chain [1] done processing
22:03:43 - cmdstanpy - INFO - Chain [1] start processing
22:03:44 - cmdstanpy - INFO - Chain [1] done processing
22:03:44 - cmdstanpy - INFO - Chain [1] start processing
22:03:45 - cmdstanpy - INFO - Chain [1] done processing
22:03:45 - cmdstanpy - INFO - Chain [1] start processing
22:03:45 - cmdstanpy - INFO - Chain [1] done processing
22:03:45 - cmdstanpy - INFO - Chain [1] start processing
22:03:46 - cmdstanpy - INFO - Chain [1] done processing
22:03:46 - cmdstanpy - INFO - Chain [1] start processing
22:03:46 - cmdstanpy - INFO - Chain [1] done processing
22:03:47 - cmdstanpy - INFO - Chain [1] start processing
22:03:47 - cmdstanpy - INFO - Chain [1]

In [10]:
print("Cross-validation performance metrics for US overall data:")
print(df_p.iloc[63])

Cross-validation performance metrics for US overall data:
horizon     365 days 00:00:00
mse                  8.819897
rmse                 2.969831
mae                  1.198219
mape                 0.003157
mdape                0.001204
smape                0.003186
coverage                 0.96
Name: 63, dtype: object


MDAPE value interpretation: https://stephenallwright.com/mdape/

Advantages of using MDAPE
- Error is returned as a percentage, making it easy to understand
- Possible to compare against other models as it’s returned as a percentage
- Not as sensitive to outliers as MAPE due it being the median

Disadvantages of using MDAPE
- If your actual values can be zero or close to zero, then MDAPE won't be possible to calculate